In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
import numpy as np
import os


In [3]:
# Class for each small NN model
class MyModelA(nn.Module):
    def __init__(self):
        super(MyModelA, self).__init__()
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
       
# Main Combined Model
# Class that uses all the small models combined and generated height , later generates all the long bones from this height
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, modelC, modelD, modelH2LL, modelH2UL, modelH2LA, modelH2UA):
        super(MyEnsemble, self).__init__()
        
        # Models that are used to predict height from detected long bone
        self.model_lower_leg = modelA
        self.model_upper_leg = modelB
        self.model_lower_arm = modelC
        self.model_upper_arm = modelD
        
        # Models for Virtual Reconstruction from height to other segments
        # Each model uses height information to predict other long bones
        
        self.height_to_lowerleg = modelH2LL
        self.height_to_upper_leg = modelH2UL
        self.height_to_lower_arm = modelH2LA
        self.height_to_upper_arm = modelH2UA
        

                
    def forward(self, x1, flag):
        
        # Function definition for height to long bone reconstruction
        def reconstruct_Segments(height):
            
                        
            # Multipication factors are the values that were used to scale values in range (0-1)
            # Every model had different dataset, therefore also had different scaling factors
            
            pred_LL  = self.height_to_lowerleg(height/201.513) * 49.319  # 201.513741064 493.195872265
            pred_UL = self.height_to_upper_leg(height/201.913) * 58.554   # 201.913788186 585.546772618
            pred_LA = self.height_to_lower_arm(height/204.805) * 31.670  # 204.805580317 316.702168533
            pred_UA = self.height_to_upper_arm(height/204.104) * 42.5100   # 204.104986515 425.100857276
            
            #returns all the predicted long bones from height
            return pred_LL, pred_UL, pred_LA, pred_UA
        
        # Check the measured/detected long bone according to recieved flag and reconstruct height from respective long bone
        # Multipication factors are the scaling factors used before training each network, results in original value.
        if flag == 'lower_leg':
            x = self.model_lower_leg(x1)
            x*=195.5    
            
        if flag == 'upper_leg':
            x = self.model_upper_leg(x1)
            x*=204.1
            
        if flag == 'lower_arm':
            x = self.model_lower_arm(x1)
            x*=204.68
            
        if flag == 'upper_arm':
            x = self.model_upper_arm(x1)
            x*= 209.2665
            
        #print('Height predicted from {} : is {} cm'.format(flag, x.item()))
        # After reconstructing height, pass the height to reconstruction function to reconstruct all the long bones form height.
        pred_LL, pred_UL, pred_LA, pred_UA = reconstruct_Segments(x)
        
        return x, pred_LL, pred_UL, pred_LA, pred_UA


In [1]:
# Create models and load state_dicts    

# Initialising 4 models for each long bone to reconstruct height

model_lower_leg = MyModelA()
model_upper_leg = MyModelA()
model_lower_arm = MyModelA()
model_upper_arm = MyModelA()


#Virtual Segments Reconstruction Model
#Initialising Models that will predict each long bone from heigh

model_height_to_lowerleg = MyModelA()
model_height_to_upper_leg = MyModelA()
model_height_to_lower_arm = MyModelA()
model_height_to_upper_arm = MyModelA()

# Load state dicts
# Paths to weights for each long bone models
PATH_lower_leg = 'Trained_models/lower_leg_pytorch_/ansur_model_2.6701'
PATH_upper_leg = 'Trained_models/upper_leg_model_pytorch_/upper_leg_5.84564208984375'
PATH_lower_arm = 'Trained_models/lower_arm_pytorch_/lower_arm_pytorch_3.489548921585083'
PATH_upper_arm = 'Trained_models/upper_arm_pytorch_/upper_arm_pytorch4.744106292724609'

# Virtual Models
# Paths to weights for each virtual height to long bone models
PATH_model_height_to_lowerleg =  'Virtual_Segments/Trained_models/height_lower_leg_5.075006484985352'
PATH_model_height_to_upper_leg = 'Virtual_Segments/Trained_models/height_upper_leg_18.44318962097168'
PATH_model_height_to_lower_arm = 'Virtual_Segments/Trained_models/height_lower_arm_9.933013916015625'
PATH_model_height_to_upper_arm = 'Virtual_Segments/Trained_models/Height_upper_arm_9.007071495056152'

# Loading Learned Weights for each long bone to height models

model_lower_leg.load_state_dict(torch.load(PATH_lower_leg))
model_upper_leg.load_state_dict(torch.load(PATH_upper_leg))
model_lower_arm.load_state_dict(torch.load(PATH_lower_arm))
model_upper_arm.load_state_dict(torch.load(PATH_upper_arm))

# Loading Learned Weights for height to each Virtual Segments
model_height_to_lowerleg.load_state_dict(torch.load(PATH_model_height_to_lowerleg))
model_height_to_upper_leg.load_state_dict(torch.load(PATH_model_height_to_upper_leg))
model_height_to_lower_arm.load_state_dict(torch.load(PATH_model_height_to_lower_arm))
model_height_to_upper_arm.load_state_dict(torch.load(PATH_model_height_to_upper_arm))


#Creating the main (Combined) model with all the pre-trained loaded weights
model = MyEnsemble(model_lower_leg, model_upper_leg, model_lower_arm, model_upper_arm,
                    model_height_to_lowerleg, model_height_to_upper_leg,
                    model_height_to_lower_arm, model_height_to_upper_arm)


#  Function to Display Outputs with each long bone 

def display(output):
    x, pred_LL, pred_UL, pred_LA, pred_UA = output
    
    print('Stature: ', x.item(), 'LL: ', pred_LL.item(),'UL: ', pred_UL.item(),'LA: ', pred_LA.item(),'UA: ', pred_UA.item())
    


NameError: name 'MyModelA' is not defined

In [13]:
## Inference Using Knee_Height
# Test value 
lower_leg = 511.0
flag = 'lower_leg' #176.88

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([lower_leg], dtype='float32'))

# Visualising in cm
print('Input lower_leg :', x1.item()/10)

# Reshaping to networks input size
x1.view((1,1))

# Passing through the main model with flag to generate height and also other segments
output = model(x1/592.0, flag)

display(output)

Input lower_leg : 51.1
Height predicted from lower_leg : is 177.85983276367188 cm
Stature:  177.85983276367188 LL:  40.69742965698242 UL:  48.497840881347656 LA:  24.779220581054688 UA:  34.680152893066406


In [9]:
## Inference Using upper_leg
# Test value 
upper_leg = 44.0
flag = 'upper_leg'  # 174.8

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([upper_leg], dtype='float32'))

# Printing input value in cm
print('\nInput upper_leg :', x1.item())


# Reshaping to networks input size
x1.view((1,1))        

# Passing through the main model with flag to generate height and also other segments
output = model(x1/55.5, flag)  

display(output)



Input upper_leg : 44.0
Height predicted from upper_leg : is 176.82974243164062 cm
Stature:  176.82974243164062 LL:  40.39461135864258 UL:  48.42181396484375 LA:  24.714984893798828 UA:  34.527488708496094


In [10]:

## Inference Using Lower Arm
# Test value 
lower_arm = 266.52
flag = 'lower_arm'  # 192.076

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([lower_arm], dtype='float32'))

# Printing input value in cm
print('\nInput lower_arm :', x1.item())

# Reshaping to networks input size
x1.view((1,1))        

# Passing through the main model with flag to generate height and also other segments
output = model(x1/280.15, flag)

display(output)



Input lower_arm : 266.5199890136719
Height predicted from lower_arm : is 189.4103546142578 cm
Stature:  189.4103546142578 LL:  44.09303665161133 UL:  49.350345611572266 LA:  25.499509811401367 UA:  36.301734924316406


In [11]:
## Inference Using Sitting Height Under 18
# Test value 
upper_arm = 358.96
flag = 'upper_arm'  # 182.67

# Converting value to a pytorch tensor
x1 = torch.from_numpy(np.array([upper_arm], dtype='float32'))

# Printing input value in cm
print('\nInput upper_arm :', x1.item())

# Reshaping to networks input size
x1.view((1,1))        

# Passing through the main model with flag to generate height and also other segments
output = model(x1/389.098, flag)
display(output)


Input upper_arm : 358.9599914550781
Height predicted from upper_arm : is 181.39915466308594 cm
Stature:  181.39915466308594 LL:  41.737918853759766 UL:  48.759063720703125 LA:  24.99993133544922 UA:  35.2047004699707
